In [1]:
import sqlite3

# SQLite 데이터베이스에 연결 (데이터베이스가 없으면 새로 생성)
conn = sqlite3.connect("sqlite3.db")
cursor = conn.cursor()

# 예시 테이블 생성 (필요에 따라 테이블 스키마를 정의하세요)
cursor.execute("""
CREATE TABLE IF NOT EXISTS interactions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_input TEXT,
    bot_response TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
)
""")
conn.commit()

In [2]:
from langchain.chat_models import ChatOpenAI

from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")
# LangChain OpenAI 모델 생성 (API 키 필요)
llm = ChatOpenAI(temperature=0.1)

In [3]:
from langchain.memory import ConversationKGMemory

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

In [4]:
user_input = "Hi I'm hamin, I live in South Korea"
bot_response = "Wow that is so cool!"

In [5]:
add_message(user_input, bot_response)

In [6]:
def save_interaction(user_input, bot_response):
    # 데이터베이스에 데이터 저장
    cursor.execute("""
    INSERT INTO interactions (user_input, bot_response) 
    VALUES (?, ?)
    """, (user_input, bot_response))
    conn.commit()
    print("Data saved successfully.")


In [7]:
# 사용자 입력과 LangChain 응답을 저장
save_interaction(user_input, bot_response)

Data saved successfully.


In [8]:
user_input = "hamin likes kimchi"
bot_response = "I like Kimchi too"
add_message(user_input, bot_response)

# 사용자 입력과 LangChain 응답을 저장
save_interaction(user_input, bot_response)

Data saved successfully.


In [9]:
def fetch_all_interactions():
    cursor.execute("SELECT * FROM interactions")
    rows = cursor.fetchall()
    for row in rows:
        print(row)

In [10]:
# 데이터 조회
fetch_all_interactions()

(1, "Hi I'm hamin, I live in South Korea", 'Wow that is so cool!', '2024-10-29 17:56:03')
(2, "Hi I'm hamin, I live in South Korea", 'Wow that is so cool!', '2024-10-29 18:01:42')
(3, 'hamin likes kimchi', 'I like Kimchi too', '2024-10-29 18:01:47')
